In [3]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K


In /home/li/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/seaborn-poster.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In /home/li/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/seaborn-poster.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In /home/li/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/seaborn-poster.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In /home/li/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/seaborn-poster.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In /home

In [1]:
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    LSTM_layer = LSTM(n_a, return_sequences=True)
    densor = Dense(n_values, activation='softmax')

    Y = X = Input(shape=(Tx, n_values))
    Y = LSTM_layer(Y)
    Y = densor(Y)

    model = Model(inputs=X, outputs=Y)
    return model, LSTM_layer, densor


In [4]:
model, LSTM_layer, densor = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 78)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 64)            36608     
_________________________________________________________________
dense_1 (Dense)              (None, 30, 78)            5070      
Total params: 41,678
Trainable params: 41,678
Non-trainable params: 0
_________________________________________________________________


In [6]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
X, Y, n_values, indices_values = load_music_utils()
Y = Y.transpose(1, 0, 2)
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)


shape of X: (60, 30, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (60, 30, 78)


In [8]:
model.fit(X, Y, epochs=100)

Epoch 1/100
60/60 [==============================] - 3s 43ms/step - loss: 4.2003 - acc: 0.0456
Epoch 2/100
60/60 [==============================] - 0s 2ms/step - loss: 4.0691 - acc: 0.1083
Epoch 3/100
60/60 [==============================] - 0s 2ms/step - loss: 3.8707 - acc: 0.0856
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 3.7786 - acc: 0.1167
Epoch 5/100
60/60 [==============================] - 0s 2ms/step - loss: 3.7043 - acc: 0.1322
Epoch 6/100
60/60 [==============================] - 0s 2ms/step - loss: 3.6362 - acc: 0.1589
Epoch 7/100
60/60 [==============================] - 0s 3ms/step - loss: 3.5655 - acc: 0.1544
Epoch 8/100
60/60 [==============================] - 0s 3ms/step - loss: 3.5124 - acc: 0.1144
Epoch 9/100
60/60 [==============================] - 0s 3ms/step - loss: 3.3886 - acc: 0.1361
Epoch 10/100
60/60 [==============================] - 0s 3ms/step - loss: 3.3048 - acc: 0.1444
Epoch 11/100
60/60 [==============================] - 0s 3

In [13]:
def generate_sample_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    x = x0 = Input(shape=(1, n_values), name='x0')
    a = a0 = Input(shape=(n_a, ), name='a0')
    c = c0 = Input(shape=(n_a, ), name='c0')
    LSTM_cell = LSTM(n_a, return_state = True, name='LSTM_cell')
    # densor = Dense(n_values, activation='softmax', name='softmax')
 
    outputs = []

    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        y = densor(a)
        outputs.append(y)
        x = Lambda(one_hot)(y)

    sample_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    return sample_model


In [14]:
sample_model = generate_sample_model(LSTM_layer, densor)
# sample_model.compile()
sample_model.get_layer(name='LSTM_cell').set_weights(LSTM_layer.get_weights())
# sample_model.get_layer(name='softmax').set_weights(dense_layer.get_weights())
# x_initializer = np.zeros((1, 1, 78))
# a_initializer = np.zeros((1, n_a))
# c_initializer = np.zeros((1, n_a))
# sample_model.predict([x_initializer, a_initializer, c_initializer])


In [16]:
out_stream = generate_music(sample_model)

Predicting new values for different set of chords.
Generated 101 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 100 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi
